# Random Nonoverlapping Rectangles

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

Al construir cada rectángulo partiendo de su centro, dimensiones y ángulo, además de los cuatro vértices se calculan sus coordenadas homogéneas y las rectas definidas por cada lado mediante cross products, que por la orientación elegida dejan el interior del rectángulo como negativo.

Con este rectángulo "aumentado" la función `disjoint` simplemente comprueba si alguna recta deja positivos los 4 vértices del otro rectángulo. Hay que hacerlo en las dos direcciones.

In [ ]:
square = np.array([[-1,-1],[-1,1],[1,1],[1,-1],[-1,-1]])

def rot(angle):
    c = np.cos(angle);
    s = np.sin(angle);
    return np.array([[c,-s],[s,c]])


class Rectangle:
    def __init__(self, x, y, width, height, angle_deg):
        self.x = x
        self.y = y
        self.width = width
        self.height = height
        self.angle_deg = angle_deg
        aux = square * (width/2, height/2)
        self.pts = aux @ rot(np.radians(angle_deg)).T + (x,y)
        X,Y = self.pts.T
        self.hpts = np.hstack([self.pts[:4],np.ones((4,1))])
        self.lines = np.array( [np.cross([X[k],Y[k],1],[X[k+1],Y[k+1],1]) for k in range(4)] )


def disjoint(r1,r2):

    def ok_line_pts(l,ps):
        return np.all(l @ ps.T > 0)

    def ok(lr1,pr2):
        return np.any([ok_line_pts(l,pr2.hpts) for l in lr1.lines])

    return ok(r1,r2) or ok(r2,r1)

El generador de configuraciones recibe una función que produce rectángulos aleatorios que se define por separado.

In [ ]:
def generate(n, limit, random_rectangle):
    good = []
    for _ in range(limit):
        if len(good) == n:
            break

        r = random_rectangle()

        allok = True
        for t in good:
            if not disjoint(r,t):
                allok = False
                break
        if allok:
            good.append(r)

    return good

In [ ]:
def mkRandomRectangle(X,Y,W,H):
    def fun():
        x = np.random.randint(X)
        y = np.random.randint(Y)
        w = np.random.randint(1,W+1)
        h = np.random.randint(1,H+1)
        a = np.random.randint(0,360)
        return Rectangle(x,y,w,h,a)
    return fun

In [ ]:
rects = generate(50,200, mkRandomRectangle(20,20,5,5))

plt.figure(figsize=(8,8))
plt.axis('equal')

for r in rects:
    plt.fill(*r.pts.T, color='lightgray')
    plt.plot(*r.pts.T, color='black')